# 汇率预测机器人

In [1]:
import numpy as np
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

class CurrencyLSTMModel:
    def __init__(self, currency_name, data_path, look_back=1):
        self.currency_name = currency_name
        self.data_path = data_path
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def load_and_prepare_data(self):
        dataset = read_csv(self.data_path, header=0, index_col=0)
        values = dataset.values.astype('float32')
        scaled = self.scaler.fit_transform(values)
        
        reframed = series_to_supervised(scaled, self.look_back, 1)
        values = reframed.values
        n_train = int(len(values) * 0.7)
        train = values[:n_train, :]
        test = values[n_train:, :]
        
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        
        train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
        test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))
        
        return train_X, train_y, test_X, test_y

    def build_model(self, input_shape):
        model = Sequential()
        model.add(LSTM(50, input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(LSTM(100, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(LSTM(50, return_sequences=False, kernel_regularizer=l2(0.01)))
        model.add(Dropout(0.3))  # Dropout to prevent overfitting
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        self.model = model

    def train(self, epochs=100, batch_size=60):
        train_X, train_y, test_X, test_y = self.load_and_prepare_data()
        self.build_model((train_X.shape[1], train_X.shape[2]))

        # Early stopping and learning rate scheduler
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

        # Train the model with early stopping and learning rate scheduling
        history = self.model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_X, test_y), verbose=2, shuffle=False,
                                 callbacks=[early_stopping, lr_scheduler])
        return history

    def save(self, model_path):
        self.model.save(model_path)

    def load(self, model_path):
        self.model = load_model(model_path)

    def predict(self, X):
        yhat = self.model.predict(X)
        return yhat



2025-07-10 15:48:13.100755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 15:48:13.115484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752133693.134551   30997 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752133693.138941   30997 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752133693.151237   30997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 新币汇率预测

In [2]:
import numpy as np
import pandas as pd
import joblib


def predict(name:str):
    #CNY_HKD_to_exchange_rate.csv
    model = CurrencyLSTMModel(currency_name=name, data_path=f"CNY_{name}_to_exchange_rate.csv", look_back=100)
    model.train()
    model.save(f"{name}_model.keras")
    model.load(f"{name}_model.keras")
    joblib.dump(model.scaler, 'scaler.save')
    model.scaler = joblib.load('scaler.save')
    csvname = f"CNY_{name}_to_exchange_rate.csv"
    # 读取原始数据
    dataset = pd.read_csv(csvname, header=0, index_col=0)
    values = dataset.values.astype('float32')

    # 归一化
    scaled = model.scaler.transform(values)

    # 取最近100天
    last_100 = scaled[-100:]  # shape: (100, 1)

    # 递归预测未来20天
    future_steps = 20
    future_preds = []

    input_seq = last_100.reshape((1, 100, 1))

    for _ in range(future_steps):
        yhat = model.predict(input_seq)
        future_preds.append(yhat[0, 0])
        # 更新输入序列：去掉最早一天，加上最新预测
        new_input = np.append(input_seq[0, 1:, 0], yhat[0, 0])
        input_seq = new_input.reshape((1, 100, 1))

    # 反归一化
    future_preds = np.array(future_preds).reshape(-1, 1)
    future_real = model.scaler.inverse_transform(future_preds)
    print("未来20天预测汇率：")
    print(future_real.flatten())
    

#targets = ["JPY", "KRW", "SGD", "THB", "HKD", "MYR"]  # 多个目标货币
targets = ["SGD"]  # 多个目标货币
for target in targets:
    print(f"预测 {target} 对人民币的汇率：")
    predict(target)
    print("-" * 30)

预测 SGD 对人民币的汇率：
Epoch 1/100


2025-07-10 15:48:15.125543: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 3s - 2s/step - loss: 2.5170 - val_loss: 2.8674 - learning_rate: 1.0000e-03
Epoch 2/100
2/2 - 0s - 182ms/step - loss: 2.3040 - val_loss: 2.4526 - learning_rate: 1.0000e-03
Epoch 3/100
2/2 - 0s - 169ms/step - loss: 2.1708 - val_loss: 2.1215 - learning_rate: 1.0000e-03
Epoch 4/100
2/2 - 0s - 168ms/step - loss: 2.1152 - val_loss: 2.0246 - learning_rate: 1.0000e-03
Epoch 5/100
2/2 - 0s - 166ms/step - loss: 2.0259 - val_loss: 2.0437 - learning_rate: 1.0000e-03
Epoch 6/100
2/2 - 0s - 196ms/step - loss: 1.9266 - val_loss: 2.0577 - learning_rate: 1.0000e-03
Epoch 7/100
2/2 - 0s - 194ms/step - loss: 1.8462 - val_loss: 2.0356 - learning_rate: 1.0000e-03
Epoch 8/100
2/2 - 0s - 194ms/step - loss: 1.7673 - val_loss: 1.9721 - learning_rate: 1.0000e-03
Epoch 9/100
2/2 - 0s - 195ms/step - loss: 1.6909 - val_loss: 1.8713 - learning_rate: 1.0000e-03
Epoch 10/100
2/2 - 0s - 197ms/step - loss: 1.6111 - val_loss: 1.7527 - learning_rate: 1.0000e-03
Epoch 11/100
2/2 - 0s - 193ms/step - loss: 1.5523 - va

In [3]:
predict("MYR")

Epoch 1/100


/home/rprp/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 - 3s - 783ms/step - loss: 2.6043 - val_loss: 2.2752 - learning_rate: 1.0000e-03
Epoch 2/100
4/4 - 1s - 159ms/step - loss: 2.1471 - val_loss: 1.9719 - learning_rate: 1.0000e-03
Epoch 3/100
4/4 - 1s - 148ms/step - loss: 1.9679 - val_loss: 1.9837 - learning_rate: 1.0000e-03
Epoch 4/100
4/4 - 1s - 143ms/step - loss: 1.7997 - val_loss: 1.6819 - learning_rate: 1.0000e-03
Epoch 5/100
4/4 - 1s - 158ms/step - loss: 1.7102 - val_loss: 1.5750 - learning_rate: 1.0000e-03
Epoch 6/100
4/4 - 1s - 158ms/step - loss: 1.5398 - val_loss: 1.5728 - learning_rate: 1.0000e-03
Epoch 7/100
4/4 - 1s - 163ms/step - loss: 1.4150 - val_loss: 1.3783 - learning_rate: 1.0000e-03
Epoch 8/100
4/4 - 1s - 179ms/step - loss: 1.3097 - val_loss: 1.2333 - learning_rate: 1.0000e-03
Epoch 9/100
4/4 - 1s - 179ms/step - loss: 1.2205 - val_loss: 1.2323 - learning_rate: 1.0000e-03
Epoch 10/100
4/4 - 1s - 185ms/step - loss: 1.1077 - val_loss: 1.1337 - learning_rate: 1.0000e-03
Epoch 11/100
4/4 - 1s - 179ms/step - loss: 1.0265 -